### Manipulation of PeaskDB de novo-assisted database search results of _T. weisflogii_ rot samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Multiple injections and fragmentation strategies included
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_329-T0nd-all_PEAKS_91/

/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_329-T0nd-all_PEAKS_91


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

329-T0nd-all_PEAKS_91_DB-search-psm.csv
329-T0nd-all_PEAKS_91_DNO.csv
329-T0nd-all_PEAKS_91_peptide.csv
329-T0nd-all_PEAKS_91_peptides_1_0_0.mzid
329-T0nd-all_PEAKS_91_peptides.pep.xml
329-T0nd-all_PEAKS_91_protein-peptides.csv
329-T0nd-all_PEAKS_91_proteins.csv
329-T0nd-all_PEAKS_91_proteins.fasta


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup329 = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_329-T0nd-all_PEAKS_91/329-T0nd-all_PEAKS_91_peptide.csv")

# remove redundant rows
peaksdb329 = pd.DataFrame.drop_duplicates(peaksdbdup329)

print(peaksdb329.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb329.columns = columns

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup329))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb329))

#look at the dataframe
peaksdb329.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area 329_T0_nodigest', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec 329_T0_nodigest', 'Accession', 'PTM', 'AScore'],
      dtype='object')
# redundant peaksdb peptides in combined dataframe 252
# nonredundant peaksdb peptides in combined dataframe 252


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source File,#Spec,#Spec,Accession,PTM,AScore
0,LPQVEGTGGDVQPSQDLVR,45.97,1994.0068,19,0.4,998.0111,74.70,133000000.0,16,23778,20210114_Weissrot_329_T0_nodigest_DDA_120min_2...,11,11,gi|54036848,NaN,NaN
1,VIGQNEAVDAVSNAIR,44.70,1654.8638,16,0.3,828.4394,90.49,128000000.0,16,29705,20210114_Weissrot_329_T0_nodigest_DDA_120min_2...,17,17,gi|54036848,NaN,NaN
2,DADGGDGPEAPSEEK(+14.02)N,38.95,1600.6488,16,2.2,801.3334,28.42,146000.0,16,7864,20210114_Weissrot_329_T0_nodigest_DDA_120min_2...,1,1,Thalassiosira_weissflogii_0203279580,Methylation(KR),K15:Methylation(KR):1000.00
3,GGGTIGHPDGIQA,37.88,1178.5680,13,0.9,590.2918,51.36,20100000.0,16,15501,20210114_Weissrot_329_T0_nodigest_DDA_120min_2...,3,3,tr|A8DP72|A8DP72_THAWE:Thalassiosira_weissflog...,NaN,NaN
4,GGGTIGHPDGIQ,37.38,1107.5309,12,1.0,554.7733,42.95,26100000.0,16,12694,20210114_Weissrot_329_T0_nodigest_DDA_120min_2...,4,4,tr|A8DP72|A8DP72_THAWE:Thalassiosira_weissflog...,NaN,NaN


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb329['A'] = peaksdb329['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb329['C'] = peaksdb329['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb329['D'] = peaksdb329['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb329['E'] = peaksdb329['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb329['F'] = peaksdb329['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb329['G'] = peaksdb329['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb329['H'] = peaksdb329['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb329 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb329['I'] = peaksdb329['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb329['K'] = peaksdb329['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb329['L'] = peaksdb329['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb329['M'] = peaksdb329['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb329['N'] = peaksdb329['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb329['P'] = peaksdb329['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb329['Q'] = peaksdb329['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb329['R'] = peaksdb329['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb329['S'] = peaksdb329['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb329['T'] = peaksdb329['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb329['V'] = peaksdb329['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb329['W'] = peaksdb329['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb329['Y'] = peaksdb329['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb329['c-carb'] = peaksdb329['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb329['m-oxid'] = peaksdb329['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a count function to enumerate the # of oxidized K's in each peptide
peaksdb329['k-oxid'] = peaksdb329['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of oxidized P's in each peptide
peaksdb329['p-oxid'] = peaksdb329['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of oxidized R's in each peptide
peaksdb329['r-oxid'] = peaksdb329['Peptide'].apply(lambda x: x.count('R(+15.99)'))

# use a count function to enumerate the # of oxidized Y's in each peptide
peaksdb329['y-oxid'] = peaksdb329['Peptide'].apply(lambda x: x.count('Y(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb329['n-deam'] = peaksdb329['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of methylated K's in each peptide
peaksdb329['k-meth'] = peaksdb329['Peptide'].apply(lambda x: x.count('K(+14.02)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaksdb329['r-meth'] = peaksdb329['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# use a count function to enumerate the # of pyro glu Q's in each peptide
peaksdb329['q-pyro'] = peaksdb329['Peptide'].apply(lambda x: x.count('Q(-17.03)'))

# use a count function to enumerate the # of acetylation of K's in each peptide
peaksdb329['k-acet'] = peaksdb329['Peptide'].apply(lambda x: x.count('K(+42.01)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb329['stripped peptide'] = peaksdb329['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb329['stripped length'] = peaksdb329['stripped peptide'].apply(len)

peaksdb329['NAAF num.'] = peaksdb329['Area'] / peaksdb329['stripped length']

# total the number of modifications in sequence
peaksdb329['ptm-total'] = peaksdb329['c-carb'] + peaksdb329['m-oxid'] + peaksdb329['k-oxid'] + peaksdb329['p-oxid'] \
+ peaksdb329['r-oxid'] + peaksdb329['y-oxid'] + peaksdb329['n-deam'] + peaksdb329['k-meth'] + peaksdb329['r-meth'] \
+ peaksdb329['q-pyro'] + peaksdb329['k-acet']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb329['stripped I-L']= peaksdb329['stripped peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb329.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_329_T0_trypsin_combine_PTMopt_DB_FDR1.csv")

# check out the results
peaksdb329.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,n-deam,k-meth,r-meth,q-pyro,k-acet,stripped peptide,stripped length,NAAF num.,ptm-total,stripped I-L
0,LPQVEGTGGDVQPSQDLVR,45.97,1994.0068,19,0.4,998.0111,74.70,133000000.0,16,23778,...,0,0,0,0,0,LPQVEGTGGDVQPSQDLVR,19,7.000000e+06,0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR,44.70,1654.8638,16,0.3,828.4394,90.49,128000000.0,16,29705,...,0,0,0,0,0,VIGQNEAVDAVSNAIR,16,8.000000e+06,0,VLGQNEAVDAVSNALR
2,DADGGDGPEAPSEEK(+14.02)N,38.95,1600.6488,16,2.2,801.3334,28.42,146000.0,16,7864,...,0,1,0,0,0,DADGGDGPEAPSEEKN,16,9.125000e+03,1,DADGGDGPEAPSEEKN
3,GGGTIGHPDGIQA,37.88,1178.5680,13,0.9,590.2918,51.36,20100000.0,16,15501,...,0,0,0,0,0,GGGTIGHPDGIQA,13,1.546154e+06,0,GGGTLGHPDGLQA
4,GGGTIGHPDGIQ,37.38,1107.5309,12,1.0,554.7733,42.95,26100000.0,16,12694,...,0,0,0,0,0,GGGTIGHPDGIQ,12,2.175000e+06,0,GGGTLGHPDGLQ


### Exporting txt files of stripped peptides at confidence cutoffs:

In [6]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep329moddup = peaksdb329[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep329dup = peaksdb329[["stripped peptide"]]

# deduplicate both of these lists
pep329mod = pep329moddup.drop_duplicates()
pep329 = pep329dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 329:", len(pep329moddup))
print("Deduplicated modified PeaksDB peptides in 329:", len(pep329mod))
print("Total stripped PeaksDB peptides in 329:", len(pep329dup))
print("Deduplicated stripped PeaksDB peptides in 329:", len(pep329))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep329.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_329_T0_trypsin_combine_PTMopt_DB_FDR1_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_329_T0_trypsin_combine_PTMopt_DB_FDR1_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_329_T0_trypsin_combine_PTMopt_DB_FDR1_stripped_peptides.fas

# look at the stripped peptides
pep329.head()

Total modified PeaksDB peptides in 329: 252
Deduplicated modified PeaksDB peptides in 329: 252
Total stripped PeaksDB peptides in 329: 252
Deduplicated stripped PeaksDB peptides in 329: 243


,stripped peptide
0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR
2,DADGGDGPEAPSEEKN
3,GGGTIGHPDGIQA
4,GGGTIGHPDGIQ


## NAAF correction and exporting files with AA and PTM totals:

In [7]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb329['A'].sum(),
        'C': peaksdb329['C'].sum(),
        'D': peaksdb329['D'].sum(),
        'E': peaksdb329['E'].sum(),
        'F': peaksdb329['F'].sum(),
        'G': peaksdb329['G'].sum(),
        'H': peaksdb329['H'].sum(),
        'I': peaksdb329['I'].sum(),
        'K': peaksdb329['K'].sum(),
        'L': peaksdb329['L'].sum(),
        'M': peaksdb329['M'].sum(),
        'N': peaksdb329['N'].sum(),
        'P': peaksdb329['P'].sum(),
        'Q': peaksdb329['Q'].sum(),
        'R': peaksdb329['R'].sum(),
        'S': peaksdb329['S'].sum(),
        'T': peaksdb329['T'].sum(),
        'V': peaksdb329['V'].sum(),
        'W': peaksdb329['W'].sum(),
        'Y': peaksdb329['Y'].sum(),
        'c-carb': peaksdb329['c-carb'].sum(),
        'm-oxid': peaksdb329['m-oxid'].sum(),
        'k-oxid': peaksdb329['k-oxid'].sum(),
        'p-oxid': peaksdb329['p-oxid'].sum(),
        'r-oxid': peaksdb329['r-oxid'].sum(),
        'y-oxid': peaksdb329['y-oxid'].sum(),
        'n-deam': peaksdb329['n-deam'].sum(),
        'k-meth': peaksdb329['k-meth'].sum(),
        'r-meth': peaksdb329['r-meth'].sum(),
        'q-pyro': peaksdb329['q-pyro'].sum(),
        'k-acet': peaksdb329['k-acet'].sum(),
        'Total area': peaksdb329['Area'].sum(),
        'Total length': peaksdb329['stripped length'].sum()
       }

totalpeaksdb329 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', \
                                              'y-oxid', 'n-deam', 'k-meth', 'r-meth', 'q-pyro', \
                                              'k-acet', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb329['% C w/ carb'] = totalpeaksdb329['c-carb'] / totalpeaksdb329['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb329['% M w/ oxid'] = totalpeaksdb329['m-oxid'] / totalpeaksdb329['M'] 

# calculate percentage of K's that are oxidized
totalpeaksdb329['% K w/ oxid'] = totalpeaksdb329['k-oxid'] / totalpeaksdb329['K'] 

# calculate percentage of P's that are oxidized
totalpeaksdb329['% P w/ oxid'] = totalpeaksdb329['p-oxid'] / totalpeaksdb329['P'] 

# calculate percentage of R's that are oxidized
totalpeaksdb329['% R w/ oxid'] = totalpeaksdb329['p-oxid'] / totalpeaksdb329['R'] 

# calculate percentage of Y's that are oxidized
totalpeaksdb329['% Y w/ oxid'] = totalpeaksdb329['y-oxid'] / totalpeaksdb329['Y'] 

# calculate percentage of N's that are deamidated
totalpeaksdb329['% N w/ deam'] = totalpeaksdb329['n-deam'] / totalpeaksdb329['N'] 

# calculate percentage of K's that are methylated
totalpeaksdb329['% K w/ meth'] = totalpeaksdb329['k-meth'] / totalpeaksdb329['K'] 

# calculate percentage of R's that are methylated
totalpeaksdb329['% R w/ meth'] = totalpeaksdb329['r-meth'] / totalpeaksdb329['R'] 

# calculate percentage of Q's that are pyro glu'd
totalpeaksdb329['% Q w/ pyro'] = totalpeaksdb329['q-pyro'] / totalpeaksdb329['Q'] 

# calculate percentage of K's that are acetylation
totalpeaksdb329['% K w/ acet'] = totalpeaksdb329['k-acet'] / totalpeaksdb329['K'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb329['NAAF denom.'] = totalpeaksdb329['Total area'] / totalpeaksdb329['Total length']

# write modified dataframe to new txt file
totalpeaksdb329.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_329_T0_trypsin_combine_PTMopt_DB_FDR1_totals.csv")

totalpeaksdb329.head()

,A,C,D,E,F,G,H,I,K,L,...,% K w/ oxid,% P w/ oxid,% R w/ oxid,% Y w/ oxid,% N w/ deam,% K w/ meth,% R w/ meth,% Q w/ pyro,% K w/ acet,NAAF denom.
sample total,158,0,254,201,73,411,70,177,81,139,...,0.012346,0.040956,0.173913,0.02439,0.018182,0.037037,0.0,0.067308,0.0,565305.09375


In [8]:
# use the calculated NAAF factor (in totalpeaksdb329 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 565305.09375

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb329['NAAF factor'] = (peaksdb329['NAAF num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb329_NAAF = peaksdb329[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', 'y-oxid', 'n-deam', \
                              'k-meth', 'r-meth', 'q-pyro', 'k-acet']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb329_NAAF['A-NAAF20'] = peaksdb329_NAAF['A'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['C-NAAF20'] = peaksdb329_NAAF['C'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['D-NAAF20'] = peaksdb329_NAAF['D'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['E-NAAF20'] = peaksdb329_NAAF['E'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['F-NAAF20'] = peaksdb329_NAAF['F'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['G-NAAF20'] = peaksdb329_NAAF['G'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['H-NAAF20'] = peaksdb329_NAAF['H'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['I-NAAF20'] = peaksdb329_NAAF['I'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['K-NAAF20'] = peaksdb329_NAAF['K'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['L-NAAF20'] = peaksdb329_NAAF['L'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['M-NAAF20'] = peaksdb329_NAAF['M'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['N-NAAF20'] = peaksdb329_NAAF['N'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['P-NAAF20'] = peaksdb329_NAAF['P'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['Q-NAAF20'] = peaksdb329_NAAF['Q'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['R-NAAF20'] = peaksdb329_NAAF['R'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['S-NAAF20'] = peaksdb329_NAAF['S'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['T-NAAF20'] = peaksdb329_NAAF['T'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['V-NAAF20'] = peaksdb329_NAAF['V'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['W-NAAF20'] = peaksdb329_NAAF['W'] * peaksdb329['NAAF factor']
peaksdb329_NAAF['Y-NAAF20'] = peaksdb329_NAAF['Y'] * peaksdb329['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb329_NAAF['ccarb-NAAF20'] = peaksdb329_NAAF['c-carb'] * peaksdb329_NAAF['NAAF factor']
peaksdb329_NAAF['moxid-NAAF20'] = peaksdb329_NAAF['m-oxid'] * peaksdb329_NAAF['NAAF factor']
peaksdb329_NAAF['koxid-NAAF20'] = peaksdb329_NAAF['k-oxid'] * peaksdb329_NAAF['NAAF factor']
peaksdb329_NAAF['poxid-NAAF20'] = peaksdb329_NAAF['p-oxid'] * peaksdb329_NAAF['NAAF factor']
peaksdb329_NAAF['roxid-NAAF20'] = peaksdb329_NAAF['r-oxid'] * peaksdb329_NAAF['NAAF factor']
peaksdb329_NAAF['yoxid-NAAF20'] = peaksdb329_NAAF['y-oxid'] * peaksdb329_NAAF['NAAF factor']
peaksdb329_NAAF['ndeam-NAAF20'] = peaksdb329_NAAF['n-deam'] * peaksdb329_NAAF['NAAF factor']
peaksdb329_NAAF['kmeth-NAAF20'] = peaksdb329_NAAF['k-meth'] * peaksdb329_NAAF['NAAF factor']
peaksdb329_NAAF['rmeth-NAAF20'] = peaksdb329_NAAF['r-meth'] * peaksdb329_NAAF['NAAF factor']
peaksdb329_NAAF['qpyro-NAAF20'] = peaksdb329_NAAF['q-pyro'] * peaksdb329_NAAF['NAAF factor']
peaksdb329_NAAF['kacet-NAAF20'] = peaksdb329_NAAF['k-acet'] * peaksdb329_NAAF['NAAF factor']

# write the dataframe to a new csv
peaksdb329_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_329_T0_trypsin_combine_PTMopt_DB_FDR1_NAAF.csv")

peaksdb329_NAAF.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,I,...,moxid-NAAF20,koxid-NAAF20,poxid-NAAF20,roxid-NAAF20,yoxid-NAAF20,ndeam-NAAF20,kmeth-NAAF20,rmeth-NAAF20,qpyro-NAAF20,kacet-NAAF20
0,LPQVEGTGGDVQPSQDLVR,12.382694,0,0,2,1,0,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,VIGQNEAVDAVSNAIR,14.151650,3,0,1,1,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,DADGGDGPEAPSEEKN,0.016142,2,0,3,3,0,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.016142,0.0,0.0,0.0
3,GGGTIGHPDGIQA,2.735079,1,0,1,0,0,5,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,GGGTIGHPDGIQ,3.847480,0,0,1,0,0,5,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [9]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb329 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb329_NAAF['NAAF factor'].sum(),
        'A': peaksdb329_NAAF['A'].sum(),
        'C': peaksdb329_NAAF['C'].sum(),
        'D': peaksdb329_NAAF['D'].sum(),
        'E': peaksdb329_NAAF['E'].sum(),
        'F': peaksdb329_NAAF['F'].sum(),
        'G': peaksdb329_NAAF['G'].sum(),
        'H': peaksdb329_NAAF['H'].sum(),
        'I': peaksdb329_NAAF['I'].sum(),
        'K': peaksdb329_NAAF['K'].sum(),
        'L': peaksdb329_NAAF['L'].sum(),
        'M': peaksdb329_NAAF['M'].sum(),
        'N': peaksdb329_NAAF['N'].sum(),
        'P': peaksdb329_NAAF['P'].sum(),
        'Q': peaksdb329_NAAF['Q'].sum(),
        'R': peaksdb329_NAAF['R'].sum(),
        'S': peaksdb329_NAAF['S'].sum(),
        'T': peaksdb329_NAAF['T'].sum(),
        'V': peaksdb329_NAAF['V'].sum(),
        'W': peaksdb329_NAAF['W'].sum(),
        'Y': peaksdb329_NAAF['Y'].sum(),
        'c-carb': peaksdb329_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb329_NAAF['moxid-NAAF20'].sum(),
        'k-oxid': peaksdb329_NAAF['koxid-NAAF20'].sum(),
        'p-oxid': peaksdb329_NAAF['poxid-NAAF20'].sum(),
        'r-oxid': peaksdb329_NAAF['roxid-NAAF20'].sum(),
        'y-oxid': peaksdb329_NAAF['yoxid-NAAF20'].sum(),
        'n-deam': peaksdb329_NAAF['ndeam-NAAF20'].sum(),
        'k-meth': peaksdb329_NAAF['kmeth-NAAF20'].sum(),
        'r-meth': peaksdb329_NAAF['rmeth-NAAF20'].sum(),
        'q-pyro': peaksdb329_NAAF['qpyro-NAAF20'].sum(),
        'k-acet': peaksdb329_NAAF['kacet-NAAF20'].sum()
       }

totalpeaksdb329_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', \
                                                   'y-oxid', 'n-deam', 'k-meth', 'r-meth', \
                                                   'q-pyro', \
                                                   'k-acet'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb329_NAAF['% C w/ carb.'] = totalpeaksdb329_NAAF['c-carb'] / totalpeaksdb329_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb329_NAAF['% M w/ oxid'] = totalpeaksdb329_NAAF['m-oxid'] / totalpeaksdb329_NAAF['M'] 

# calculate NAAF-corrected percentage of K's that are oxidized
totalpeaksdb329_NAAF['% K w/ oxid'] = totalpeaksdb329_NAAF['k-oxid'] / totalpeaksdb329_NAAF['K'] 

# calculate NAAF-corrected percentage of P's that are oxidized
totalpeaksdb329_NAAF['% P w/ oxid'] = totalpeaksdb329_NAAF['p-oxid'] / totalpeaksdb329_NAAF['P'] 

# calculate NAAF-corrected percentage of R's that are oxidized
totalpeaksdb329_NAAF['% R w/ oxid'] = totalpeaksdb329_NAAF['r-oxid'] / totalpeaksdb329_NAAF['R'] 

# calculate NAAF-corrected percentage of Y's that are oxidized
totalpeaksdb329_NAAF['% Y w/ oxid'] = totalpeaksdb329_NAAF['y-oxid'] / totalpeaksdb329_NAAF['Y'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb329_NAAF['% N w/ deam'] = totalpeaksdb329_NAAF['n-deam'] / totalpeaksdb329_NAAF['N'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaksdb329_NAAF['% K w/ meth'] = totalpeaksdb329_NAAF['k-meth'] / totalpeaksdb329_NAAF['K'] 

# calculate NAAF-corrected percentage of R's that are methylated
totalpeaksdb329_NAAF['% R w/ meth'] = totalpeaksdb329_NAAF['r-meth'] / totalpeaksdb329_NAAF['R'] 

# calculate NAAF-corrected percentage of Q's that are pyro glu'd
totalpeaksdb329_NAAF['% Q w/ pyro'] = totalpeaksdb329_NAAF['q-pyro'] / totalpeaksdb329_NAAF['Q'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaksdb329_NAAF['% K w/ acet'] = totalpeaksdb329_NAAF['k-acet'] / totalpeaksdb329_NAAF['K'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb329_NAAF['NAAF check'] = totalpeaksdb329_NAAF['NAAF'] / 565305.09375

# write modified dataframe to new txt file, same name + totals
totalpeaksdb329_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_329_T0_trypsin_combine_PTMopt_DB_FDR1_NAAF_totals.csv")

totalpeaksdb329_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,% K w/ oxid,% P w/ oxid,% R w/ oxid,% Y w/ oxid,% N w/ deam,% K w/ meth,% R w/ meth,% Q w/ pyro,% K w/ acet,NAAF check
sample total,238.206235,158,0,254,201,73,411,70,177,81,...,0.0,0.010306,0.006839,0.000224,0.001513,0.001104,0.0,0.01364,0.0,0.000421
